<a href="https://colab.research.google.com/github/fzanart/Socialz/blob/main/EA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Algorithm 1 Generate a random TSP instance.
def randomInstance(size):
  for i = 1 → size do 
    instance[i, 1] ← U(0, 1) #? Uniform random number between 0 and 1 
    instance[i, 2] ← U(0, 1) #? Uniform random number between 0 and 1
  return instance


In [ ]:
# Algorithm 2 EA for evolving problem easy and hard TSP instances
def EA(popSize, instSize, generations, time_limit, cells, repetitions, type, rnd, mutationParameters) 
  poolSize ← ?popSize/2? 
  for i = 1 → popSize do 
    population[i] ← rescale(randomInstance(instSize)) 
    population[i] ← round(population[i], cells) 
    if rnd then 
    population[i] ← normalMutation(population[i]) 
    population[i] ← CutToBoundary(population[i])

  for generation = 1 → generations do 
    for k = 1 → popSize do 
    fitness[k] ← computeFitness(population[k], repetitions)

  matingPool ← createMatingPool(poolSize, population, fitness) 
  nextPopulation[1] ← population[bestOf(fitness)] ? 1-elitism

    for k = 2 → popSize do 
    parent1 ← randomElement(matingPool) 
    parent2 ← randomElement(matingPool) 
    offspring ← uniformMutation(uniformCrossover(parent1, parent2)) 
    if !rnd then 
      offspring ← normalMutation(offspring)
    offspring ← rescale(offspring) 
    offspring ← round(offspring, cells) 
    if rnd then 
    offspring ← normalMutation(offspring) 
    offspring ← CutToBoundary(offspring)

  population ← nextPopulation 
  if over time limit time_limit then 
  return population


In [ ]:
# Algorithm 3 Compute Fitness
def ComputeFitness(instance, repetitions):
  optimalTourLength ← concorde(instance)     #CONCORDE
  for j = 1 → repetitions do 
    twoOptTourLengths[j] ← twoOpt(instance) # Two Opt Tour length

  return mean(twoOptTourLengths)/optimalTourLength



In [ ]:
# Algorithm 4 Mating pool creation
def createMatingPool(poolSize, population, fitness):
  for i = 1 → poolSize do 
  matingPool[i] ← betterOf(randomElement(population), randomElement(population))

  return matingPool
